In [1]:
import torch
from transformers import GPT2Model, GPT2Tokenizer
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader

# GPUが利用可能であれば使用
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# GPT-2モデルとトークナイザーのロード
model_name = "gpt2"
model = GPT2Model.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# WikiTextデータセットのロード
#dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
dataset = load_from_disk("wikitext-alpha-2.0")


# バッチ処理のためにデータローダーを設定
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

def embed_batch(batch):
    texts = batch["text"]
    # トークナイズし、最大シーケンス長にパディング
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # GPT-2モデルで文の埋め込みを取得
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 文の埋め込みを平均化して取得
    sentence_embeddings = outputs.last_hidden_state.mean(dim=1)
    return sentence_embeddings.cpu()

# バッチごとに文埋め込みを計算し、保存
all_embeddings = []
for batch in dataloader:
    embeddings = embed_batch(batch)
    all_embeddings.append(embeddings)

# リストをTensorに変換
all_embeddings = torch.cat(all_embeddings)

# 埋め込みを保存する (例: torch.save で保存)
#torch.save(all_embeddings, "wikitext_gpt2_embeddings.pt")
torch.save(all_embeddings, "wikitext-alpha-2.0_gpt2_embeddings.pt")

print(f"Total embeddings: {all_embeddings.size(0)}, Embedding dimension: {all_embeddings.size(1)}")



/home/shinzato/git/text_power_raw/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Total embeddings: 36718, Embedding dimension: 768
